In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import datetime as dt
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data

import os
os.listdir('formatedData/1/')

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [29]:
data = pd.read_csv("data.csv", sep=';')
labels = pd.read_csv("labels.csv",sep=',')

data1 = pd.read_csv("formatedData/1/data.csv",sep=';')
labels1 = pd.read_csv("formatedData/1/labels.txt",  skiprows=5,sep=';')

data2 = pd.read_csv("formatedData/2/data.csv",sep=';')
labels2 = pd.read_csv("formatedData/2/labels.txt",  skiprows=5,sep=';')

In [30]:
# data2

In [31]:
# labels1.columns[0]

In [32]:
## Data Date Format

def data_format(data):
    # pd.to_datetime(data['TimeStamp'], format='%d%b%Y:%H:%M:%S.%f')
    data['TimeStamp'] = data['TimeStamp'].str.slice(stop=-10)
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
    data['Date'] = [d.date() for d in data['TimeStamp']]
    data['Time'] = [d.time() for d in data['TimeStamp']]
    mid = data['Date']
    data.drop("Date", axis=1,inplace = True)
    data.insert(0, 'Date', mid)

    mid = data['Time']
    data.drop("Time", axis=1,inplace = True)
    data.insert(1, 'Time', mid)

    data.drop("TimeStamp", axis=1, inplace=True)

In [33]:
## Label Date Format
def label_Format(labels):
    
    ae = labels.columns[0]
    labels = labels.reset_index()
    labels["TimeStamp"] = labels["index"]
    labels = labels.drop("index", axis=1)
    mid = labels['TimeStamp']
    labels.drop(labels=['TimeStamp'], axis=1,inplace = True)
    labels.insert(0, 'TimeStamp', mid)
    labels["Label"] = labels[ae]
#     labels["Label"] = labels[ae].str.slice(start=-1)
    labels = labels.drop(ae, axis=1)
    # labels['TimeStamp'] = [d.time() for d in labels['TimeStamp']]
    ad = pd.to_datetime(labels['TimeStamp'])
    labels['TimeStamp'] = ad.dt.strftime('%H:%M:%S')
    return labels

In [34]:
## GET 30 Periods

def get_30s(data):
    j=0
    k=0

    begins = []

    for i in range(len(data)):
        a = data["Time"].iloc[i]

        if (a.second==0 or a.second==30):
    #         print(i-k, i)
            begins.append(i)
            j+=1
            k = i
#     print(j)
#     print(begins)
    return begins

In [35]:
####### Get Preprocesd Data Frame
def processDF(begins, data, labels):
    ####### Select Droping Features Here ####
    data1 = data.drop("Time", axis=1)
    data1 = data1.drop("Date", axis=1)
    data1 = data1.drop("FrameCounter", axis=1)
    data1 = data1.drop("State", axis=1)
      
    data_time = data.filter(["Time", "Date"], axis=1)
    avgFrame = []
    
    for i in range(len(begins)-1):
        row = []
        time = str(data_time["Time"].iloc[begins[i]])
        date = str(data_time["Date"].iloc[begins[i]])
        features = (data1.iloc[begins[i]: begins[i+1]]).mean().to_frame().transpose().values[0]
        label = labels["Label"].loc[labels['TimeStamp'] == str(data_time["Time"].iloc[begins[i]])].values[0]
        startTime = labels["TimeStamp"].loc[labels['TimeStamp'] == str(data_time["Time"].iloc[begins[i]])].values[0]
        endTime = labels["TimeStamp"].loc[labels['TimeStamp'] == str(data_time["Time"].iloc[begins[i+1]])].values[0]
        row.append(date)
    #     row.append(time)
        row.append(startTime)
        row.append(endTime)
        row.extend(features)
        row.append(label)
        avgFrame.append(row)
    #     print(date, time, features, label)
    return pd.DataFrame(avgFrame, columns=["Date", "StartTime", "EndTime", "RPM", "ObjectDistance", "SignalQuality", "MovementSlow", "MovementFast", "Label"])

In [36]:
data_format(data1)
labels = label_Format(labels1)
# label_cat = labels["Label"]
# labels["Label"] = encoder.fit_transform(label_cat)
begins = get_30s(data1)
df = processDF(begins, data1, labels)

In [37]:
formatData = os.listdir('formatedData')
formatData.remove('1')
formatData.remove('4')

In [38]:
print("Folder {:d} has been added!...... Count = {:d}".format(1, 1))
count = 1
for i in formatData[1:]:
    count+=1
#     print(os.listdir('formatedData/'+i))
    data = pd.read_csv('formatedData/' + i + '/data.csv',sep=';')
    labels = pd.read_csv("formatedData/" + i + "/labels.txt",  skiprows=5,sep=';')
    
    data_format(data)
    labels = label_Format(labels)
#     label_cat = labels["Label"]
#     labels["Label"] = encoder.fit_transform(label_cat)
    begins = get_30s(data)
    df_ = processDF(begins, data, labels)
    df = df.append(df_, ignore_index=True)
    
    print("Folder {:s} has been added!...... Count = {:d}".format(i, count))
    

#     print(labels.columns[0], count)
# print("Label Classes :", encoder.classes_ )
print("Total {:d} records has been added".format(len(df)))

Folder 1 has been added!...... Count = 1
Folder 17 has been added!...... Count = 2
Folder 32 has been added!...... Count = 3
Folder 21 has been added!...... Count = 4
Folder 39 has been added!...... Count = 5
Folder 13 has been added!...... Count = 6
Folder 20 has been added!...... Count = 7
Folder 22 has been added!...... Count = 8
Folder 41 has been added!...... Count = 9
Folder 19 has been added!...... Count = 10
Folder 34 has been added!...... Count = 11
Folder 10 has been added!...... Count = 12
Folder 8 has been added!...... Count = 13
Folder 40 has been added!...... Count = 14
Folder 29 has been added!...... Count = 15
Folder 23 has been added!...... Count = 16
Folder 3 has been added!...... Count = 17
Folder 35 has been added!...... Count = 18
Folder 37 has been added!...... Count = 19
Folder 9 has been added!...... Count = 20
Folder 27 has been added!...... Count = 21
Folder 33 has been added!...... Count = 22
Folder 43 has been added!...... Count = 23
Folder 30 has been added

In [39]:
df.head()

,Date,StartTime,EndTime,RPM,ObjectDistance,SignalQuality,MovementSlow,MovementFast,Label
0,2017-11-22,23:45:00,23:45:30,0.00000,1.005013,4.233333,17.183042,1.032979,Wake
1,2017-11-22,23:45:30,23:46:00,0.00000,0.718234,5.433333,39.918807,38.216497,Wake
2,2017-11-22,23:46:00,23:46:30,8.11255,0.552008,8.400000,66.798640,29.113056,Wake
3,2017-11-22,23:46:30,23:47:00,0.00000,0.547259,4.900000,39.094203,23.586217,Wake
4,2017-11-22,23:47:00,23:47:30,0.00000,0.410558,9.366667,71.531260,35.720941,Wake


In [40]:
df.describe()

,RPM,ObjectDistance,SignalQuality,MovementSlow,MovementFast
count,37252.000000,37252.000000,37252.000000,37252.000000,37252.000000
mean,12.599632,0.808516,8.109587,11.503253,2.132584
std,4.564791,0.315169,2.409716,14.000889,7.153354
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.994466,0.611686,6.900000,4.530837,0.000000
50%,13.705408,0.750553,9.133333,6.940323,0.009613
75%,15.703198,0.915402,10.000000,11.242233,0.206300
max,29.790257,4.860523,10.000000,96.306997,83.985140


In [64]:
df.to_csv(r'out/prepared.csv')

In [44]:
df_ = df.copy()

In [48]:
df["Label"].value_counts()

 N2          15454
 REM          8008
 N3           7800
 Wake         3888
 N1           2061
 Artefact       39
 A               2
Name: Label, dtype: int64

In [ ]:
df_['Label'] = df_['Label'].replace({'A': ' Artefact'})

In [ ]:
df

In [25]:
df_cat = df["Label"]
df["Label"] = encoder.fit_transform(df_cat)

In [27]:
encoder.classes_

array([' A', ' Artefact', ' N1', ' N2', ' N3', ' REM', ' Wake'],
      dtype=object)